In [1]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os, shutil
import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras import models
from keras import layers
from sklearn.utils import class_weight

# Baseline Model

In [2]:
# Create a data generator
# data augmentation
datagen2 = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=20, # randomly rotate images in the range
    zoom_range=0.2, # randomly zoom image 
    width_shift_range=0.2, # randomly shift images horizontally
    height_shift_range=0.2, # randomly shift images vertically 
    horizontal_flip=True) # randomly flip images

In [3]:
# Use flow_from_directory to read images from folders
train_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/train', # point to the parent directory
        target_size=(299, 299), # resize images to 299x299
        batch_size=32,
        class_mode='binary') # binary for two classes

# similarly create validation and test generators
val_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/val',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')

test_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/test',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')


Found 4098 images belonging to 2 classes.
Found 879 images belonging to 2 classes.
Found 879 images belonging to 2 classes.


In [4]:
#this is the model with dense and conv layers

model2 = models.Sequential()
model2.add(layers.Conv2D(32, (5, 5), activation='relu',
                        input_shape=(299 , 299,  3)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(32, (4, 4), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

Metal device set to: Apple M2 Pro


2023-07-12 16:08:50.191708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-12 16:08:50.191926: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
from collections import Counter

# this code creates class weights to address the data being imbalanced
# counter counts the pneumonia vs normal images (1 being pneumonia)
# the next line creates a dictionary assigning a weight to the NORMAL class (which is 0) 
# this weight for the NORMAL class is the number of PNEUMONIA images divided by the number of NORMAL images

counter = Counter(train_generator2.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

print(counter)
print(max_val)
print(class_weights)

Counter({1: 2991, 0: 1107})
2991.0
{0: 2.7018970189701896, 1: 1.0}


In [7]:
# this is where the model is trained
# the np.ceil part just makes sure that all the data (in batches) is being trained for each epoch
# class weights is applied to deal with the imbalance in the dataset

epochs = 30

steps_per_epoch2 = np.ceil(train_generator2.samples / train_generator2.batch_size)
validation_steps2 = np.ceil(val_generator2.samples / val_generator2.batch_size)

history = model2.fit(
    train_generator2,
    steps_per_epoch=int(steps_per_epoch2),
    epochs=epochs,
    validation_data=val_generator2,
    validation_steps=int(validation_steps2),
    class_weight=class_weights)



Epoch 1/30


2023-07-12 16:09:10.718813: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-12 16:09:10.895561: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - ETA: 0s - loss: 0.7777 - acc: 0.7028

2023-07-12 16:10:09.113021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - 71s 544ms/step - loss: 0.7777 - acc: 0.7028 - val_loss: 0.3614 - val_acc: 0.8737
Epoch 2/30
129/129 [==============================] - 71s 549ms/step - loss: 0.5451 - acc: 0.8404 - val_loss: 0.4468 - val_acc: 0.8589
Epoch 3/30
129/129 [==============================] - 70s 542ms/step - loss: 0.5111 - acc: 0.8594 - val_loss: 0.2683 - val_acc: 0.8817
Epoch 4/30
129/129 [==============================] - 72s 556ms/step - loss: 0.4869 - acc: 0.8638 - val_loss: 0.2568 - val_acc: 0.8896
Epoch 5/30
129/129 [==============================] - 71s 547ms/step - loss: 0.4617 - acc: 0.8626 - val_loss: 0.2662 - val_acc: 0.8931
Epoch 6/30
129/129 [==============================] - 69s 536ms/step - loss: 0.4380 - acc: 0.8777 - val_loss: 0.2292 - val_acc: 0.9067
Epoch 7/30
129/129 [==============================] - 68s 529ms/step - loss: 0.4458 - acc: 0.8804 - val_loss: 0.4594 - val_acc: 0.8316
Epoch 8/30
129/129 [==============================] - 68s 529ms/st

In [9]:
results_train = model2.evaluate(train_generator2)
results_test = model2.evaluate(val_generator2)
print('Training set performance:', results_train)
print('Validation set performance:', results_test)

28/28 [==============================] - 13s 446ms/step - loss: 0.1676 - acc: 0.9340
Training set performance: [0.17489254474639893, 0.9331381320953369]
Validation set performance: [0.16764609515666962, 0.9340159893035889]


In [11]:
model2.save('modelv1weighted.h5')  # creates a HDF5 file for the model